In [1]:
#from tkinter import SEPARATOR
import pandas as pd
import os
import tensorflow as tf
from tensorflow import keras
from keras import optimizers
from keras.regularizers import l2

In [18]:
#data path


#Load the data file into a Pandas Dataframe
symptom_data = pd.read_excel("Samlet_data.xlsx")
#names = ["Person", "Fz", "Cz", "Pz", "C3", "T3", "C4", "T4", "Fp1", "Fp2", "F3", "F4", "F7", "F8", "P3", "P4", "T5", "T6", "O1", "O2", "ADHD"]

#Explore the data loaded
print(symptom_data.dtypes)
symptom_data.head()


Person      int64
Fz        float64
Cz        float64
Pz        float64
C3        float64
T3        float64
C4        float64
T4        float64
Fp1       float64
Fp2       float64
F3        float64
F4        float64
F7        float64
F8        float64
P3        float64
P4        float64
T5        float64
T6        float64
O1        float64
O2        float64
ADHD        int64
dtype: object


,Person,Fz,Cz,Pz,C3,T3,C4,T4,Fp1,Fp2,...,F4,F7,F8,P3,P4,T5,T6,O1,O2,ADHD
0,1,406.245216,278.553994,311.195205,231.255983,182.955381,163.578897,195.899262,214.960414,269.520053,...,166.977800,108.628261,145.869417,162.385593,185.066248,111.396126,455.738872,164.177427,168.898866,1
1,2,338.874206,321.806165,145.229491,128.877279,136.471658,120.199143,120.062534,114.842249,89.742555,...,141.277772,110.007362,86.789567,122.920235,79.270314,76.847722,146.568720,121.082799,120.554203,1
2,3,432.093213,534.124105,253.705071,258.938625,263.050806,234.238388,290.515299,231.549914,228.616968,...,203.028227,194.627793,233.775727,235.030187,221.558224,236.490592,239.846506,247.045629,291.176679,1
3,4,276.289074,248.060313,189.970392,189.830029,201.918945,178.838227,209.549240,174.300613,703.473139,...,137.053173,138.588268,140.688885,108.969814,109.844978,99.918767,199.793133,149.711277,156.627688,1
4,5,223.942213,206.882842,177.814803,161.306944,189.485906,157.751333,188.326250,174.606640,136.620136,...,183.765303,141.684913,132.586913,148.385760,126.067493,124.801694,157.602355,151.765398,161.908581,1


In [19]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

label_encoder = preprocessing.LabelEncoder()
symptom_data['ADHD'] = label_encoder.fit_transform(
                                symptom_data['ADHD'])

#Convert Pandas DataFrame to a numpy vector
np_symptom = symptom_data.to_numpy().astype(float)

#Extract the feature variables (X)
X_data = np_symptom[:,1:19]

#Extract the target variable (Y), conver to one-hot-encodign
Y_data=np_symptom[:,19]

Y_data = tf.keras.utils.to_categorical(Y_data)

#Split training and test data
X_train,X_test,Y_train,Y_test = train_test_split( X_data, Y_data, test_size=0.10)

print("Shape of feature variables :", X_train.shape)
print("Shape of target variable :",Y_train.shape)


Shape of feature variables : (81, 18)
Shape of target variable : (81, 592)


In [20]:
#Setup Training Parameters
EPOCHS=20
BATCH_SIZE=64
VERBOSE=1
OUTPUT_CLASSES=len(label_encoder.classes_)
N_HIDDEN=128
VALIDATION_SPLIT=0.2

In [21]:
#Create a Keras sequential model
model = tf.keras.models.Sequential()
#Add a Dense Layer
model.add(keras.layers.Dense(N_HIDDEN,
                             input_shape=(18,),
                              name='Dense-Layer-1',
                              activation='relu'))

#Add a second dense layer
model.add(keras.layers.Dense(N_HIDDEN,
                              name='Dense-Layer-2',
                              activation='relu'))

#Add a softmax layer for categorial prediction
model.add(keras.layers.Dense(OUTPUT_CLASSES,
                             name='Final',
                             activation='softmax'))
#Compile the model
model.compile(
              loss='categorical_crossentropy',
              metrics=['accuracy'])


model.summary()


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Dense-Layer-1 (Dense)       (None, 128)               2432      
                                                                 
 Dense-Layer-2 (Dense)       (None, 128)               16512     
                                                                 
 Final (Dense)               (None, 2)                 258       
                                                                 
Total params: 19,202
Trainable params: 19,202
Non-trainable params: 0
_________________________________________________________________


In [22]:
#Build the model
model.fit(X_train,
          Y_train,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          verbose=VERBOSE,
          validation_split=VALIDATION_SPLIT)

#Evaluate the model against the test dataset and print results
print("\nEvaluation against Test Dataset :\n------------------------------------")
model.evaluate(X_test,Y_test)


Epoch 1/20


ValueError: in user code:

    File "C:\Python310\lib\site-packages\keras\engine\training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "C:\Python310\lib\site-packages\keras\engine\training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Python310\lib\site-packages\keras\engine\training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "C:\Python310\lib\site-packages\keras\engine\training.py", line 860, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "C:\Python310\lib\site-packages\keras\engine\training.py", line 918, in compute_loss
        return self.compiled_loss(
    File "C:\Python310\lib\site-packages\keras\engine\compile_utils.py", line 201, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    File "C:\Python310\lib\site-packages\keras\losses.py", line 141, in __call__
   